### План

1. Разбиваем весь текст на токены-слова
2. Удаляем стоп-слова. Местоимения и артикли в исключения
2. Приводим все к нижнему регистру
3. Уникализируем и подсчитываем количество каждого слова
4. Создаем df
4. К каждому слову добавляем столько раз предложение, сколько это слово встречается.
5. Определяем часть речи и добавляем в df
6. Если слово - глагол, что находим его 3 формы и списком добавляем в df. Проделываем эти операции со всеми частями речи по условию if
Если артикль, то добавляем этим атриклям в df список всех артиклей (для выбора)


### Виды упражнений
1. Определить правильный артикль
2. Определяем пропущенное слово
3. Определить правильную форму глагола. Ищем в предложении все глаголы и предлагаем 3 формы. Не, не пойдет. Времен больше намного
4. Расставить слова в правильном порядке. Перемешиваем все и даем выбрать





In [2]:
import pandas as pd
import nltk
import re
import transformers
#from tensorflow.python.keras.models import transformers
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, roc_curve
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier
from nltk.corpus import stopwords as nltk_stopwords
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from wordcloud import WordCloud
warnings.filterwarnings('ignore')

2023-06-26 10:37:51.129070: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [67]:
# Определяем константой типы заданий

TASKS = ['Выберите слово', 'Заполните пропуск']

In [68]:
# Загружаем каждое предложение в отдельную строку df. Ставим сепоратор таб, иначе запятая выдает ошибку
df = pd.read_csv('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt', lineterminator = '\n', header=None, sep="\t", names=['origin_sentences'])
df.head(5)

,origin_sentences
0,Little Red Cap
1,Jacob and Wilhelm Grimm
2,Once upon a time there was a sweet little girl...
3,"One day her mother said to her, ""Come Little R..."
4,Little Red Cap promised to obey her mother. Th...


In [39]:
# with open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt') as f:
#     lines = f.read()

In [65]:

# #Создаем датаврейм с каждым словом
# df_all_words = pd.DataFrame({'origin_word':nltk.word_tokenize(lines)})
# df_all_words.head(5)


In [85]:
import spacy

texts = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]

nlp = spacy.load("en_core_web_sm")
for doc in nlp.pipe(texts, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):
    # Do something with the doc here
    print([(ent.text, ent.label_) for ent in doc.ents])

[('$9.4 million', 'MONEY'), ('the prior year', 'DATE'), ('$2.7 million', 'MONEY')]
[('twelve billion dollars', 'MONEY'), ('1b', 'MONEY')]


In [69]:
tokens = []
lemma = []
pos = []

for doc in nlp.pipe(df['origin_sentences'].astype('unicode').values, batch_size=50,
                        ):
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
        
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

df['word_tokens'] = tokens
df['word_lemma'] = lemma
df['word_pos'] = pos

In [70]:
df.head(5)

,origin_sentences,word_tokens,word_lemma,word_pos
0,Little Red Cap,"[Little, Red, Cap]","[little, Red, Cap]","[ADJ, PROPN, PROPN]"
1,Jacob and Wilhelm Grimm,"[Jacob, and, Wilhelm, Grimm]","[Jacob, and, Wilhelm, Grimm]","[PROPN, CCONJ, PROPN, PROPN]"
2,Once upon a time there was a sweet little girl...,"[Once, upon, a, time, there, was, a, sweet, li...","[once, upon, a, time, there, be, a, sweet, lit...","[ADV, SCONJ, DET, NOUN, PRON, VERB, DET, ADJ, ..."
3,"One day her mother said to her, ""Come Little R...","[One, day, her, mother, said, to, her, ,, "", C...","[one, day, her, mother, say, to, she, ,, "", co...","[NUM, NOUN, PRON, NOUN, VERB, ADP, PRON, PUNCT..."
4,Little Red Cap promised to obey her mother. Th...,"[Little, Red, Cap, promised, to, obey, her, mo...","[little, Red, Cap, promise, to, obey, her, mot...","[ADJ, PROPN, PROPN, VERB, PART, VERB, PRON, NO..."


In [72]:
import pyinflect
nlp = spacy.load('en_core_web_sm')
tokens = nlp('This is an example of xxtest.')
tokens[3]._.inflect('NNS')

'examples'

In [73]:
# изменение степени прилагательного с помощью pyinflect
for token in nlp("I think it's a good idea and easy to use"):
    if word_pos=='ADJ':
        print(token.text, token._.inflect('JJS'))        

good best
easy easiest


In [84]:
item = 'ADJ'
for i in df.word_pos:
   
    if item in i:
        print(item)

ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ


In [76]:
if 'ADJ'.isin(df.word_pos):
    print('ok')

AttributeError: 'str' object has no attribute 'isin'

In [75]:


df["ADJ_forms"] = df.apply(lambda x: int(x["ride_cost"] * 0.95)
                                 if x["rating"] > 6 and x["speed_max"] < 120
                                 else int(x["ride_cost"] * 1.05),
                                 axis=1)

ValueError: Cannot set a DataFrame with multiple columns to the single column ADJ_forms

In [50]:
for word in df_all_words.origin_word:
    print('lemma:', word.lemma_)

AttributeError: 'str' object has no attribute 'lemma_'

In [60]:
# Приводим слова к начальной форме (лемме)
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(df['origin_sentences'].astype('unicode').values)

token = doc
print(token.text)
for word in doc:
    print('lemma:', word.lemma_)



ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'numpy.ndarray'>

### NLTK

In [6]:
import nltk
from nltk.corpus import stopwords

In [19]:
stops = set(stopwords.words('english'))
print(stops)

{'they', "won't", 'have', 'don', 'each', 'themselves', 'theirs', 'its', 'between', 'when', 'up', 'under', 'just', "shan't", 'again', 'here', "weren't", 'so', 'was', 'couldn', 'hasn', 'm', 'do', 'to', 'an', 'ain', 'it', 'these', 'were', 'own', 'is', 'been', 'y', "shouldn't", 'herself', 'such', 'nor', 'will', 'now', 'o', "isn't", 'didn', 'be', 'this', 'wouldn', 'his', 'my', 'i', 're', "mustn't", 'from', 'had', "couldn't", 'while', 'haven', "you'll", 'them', 'below', 'where', 'll', "didn't", 'a', 'her', 'about', 'any', 'isn', 'needn', 'in', "you've", 'your', 'our', 'against', 'too', 'shouldn', 'won', 'both', 'him', 'who', 'after', "you're", 'am', 'has', 'weren', 'myself', "haven't", 'are', 'off', 'once', 't', 'doesn', 'yourselves', "needn't", 'should', "hadn't", 'whom', "wasn't", "mightn't", 'on', 'than', 've', 'but', 'more', 'through', "aren't", 'other', 'by', 'above', 'hadn', 'and', 'doing', 'we', "hasn't", 'did', 'of', 'how', "doesn't", 'during', 'into', 'before', 'ours', 'aren', 'you'

In [17]:
# Токенизируем каждо

df['bag_of_words'] = df.origin_sentences.apply(lambda x: nltk.word_tokenize(x))

#result2 = df.assign(feature_1=df.apply(nltk.word_tokenize("a house word ")))

#bag_of_words

In [18]:
df

,origin_sentences,bag_of_words
0,Little Red Cap,"[Little, Red, Cap]"
1,Jacob and Wilhelm Grimm,"[Jacob, and, Wilhelm, Grimm]"
2,Once upon a time there was a sweet little girl...,"[Once, upon, a, time, there, was, a, sweet, li..."
3,"One day her mother said to her, ""Come Little R...","[One, day, her, mother, said, to, her, ,, ``, ..."
4,Little Red Cap promised to obey her mother. Th...,"[Little, Red, Cap, promised, to, obey, her, mo..."
5,"Good day to you, Little Red Cap.","[Good, day, to, you, ,, Little, Red, Cap, .]"
6,"Thank you, wolf.","[Thank, you, ,, wolf, .]"
7,"Where are you going so early, Little Red Cap?","[Where, are, you, going, so, early, ,, Little,..."
8,To grandmother's.,"[To, grandmother, 's, .]"
9,And what are you carrying under your apron?,"[And, what, are, you, carrying, under, your, a..."


In [1]:
df_words = pd.DataFrame({'origin_words':bag_of_words})
df_words

NameError: name 'pd' is not defined

In [29]:
# Этот код ищет словоформы, но только существвительные

from nltk.corpus import wordnet as wn

forms = set() #We'll store the derivational forms in a set to eliminate duplicates
for happy_lemma in wn.lemmas("like"): #for each "happy" lemma in WordNet
    forms.add(happy_lemma.name()) #add the lemma itself
    for related_lemma in happy_lemma.derivationally_related_forms(): #for each related lemma
        forms.add(related_lemma.name()) #add the related lemma
forms        

{'likable', 'like', 'likeable', 'likeness'}

In [5]:
tokens_sens = nltk.tokenize.sent_tokenize(text, language='english')

In [6]:
pd.set_option('display.max_colwidth', None)
df_sentences = pd.DataFrame({'sentence': tokens_sens})
df_sentences

,sentence
0,Once upon a time there was a sweet little girl.
1,"Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next."
2,Once she gave her a little cap made of red velvet.
3,"Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Cap."
4,"One day her mother said to her, ""Come Little Red Cap."
5,Here is a piece of cake and a bottle of wine.
6,Take them to your grandmother.
7,"She is sick and weak, and they will do her well."
8,Mind your manners and give her my greetings.
9,"Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother."""


In [ ]:
pd.set_option('display.max_colwidth', 50)

### Spacy

She did not know what a wicked animal he was, and was not afraid of him


In [51]:
# @Language.component("segm")
# def set_custom_segmentation(doc):
#     for token in doc[:-1]:
#         if token.text == '{S}':
#             doc[token.i+1].is_sent_start = True
#         else:
#             doc[token.i+1].is_sent_start = False
#     return doc

In [86]:
import spacy
#from spacy.lang.ru import Russian
#from spacy import displacy
#from spacy.lang.en.examples import sentences 

#import en_core_web_sm
#nlp = en_core_web_sm.load()

nlp = spacy.load("en_core_web_sm")
doc = nlp('''
Once upon a time there was a sweet little girl. Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next. Once she gave her a little cap made of red velvet. Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Cap.
One day her mother said to her, "Come Little Red Cap. Here is a piece of cake and a bottle of wine. Take them to your grandmother. She is sick and weak, and they will do her well. Mind your manners and give her my greetings. Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother."

Little Red Cap promised to obey her mother. The grandmother lived out in the woods, a half hour from the village. When Little Red Cap entered the woods a wolf came up to her. She did not know what a wicked animal he was, and was not afraid of him.

"Good day to you, Little Red Cap."

"Thank you, wolf."

"Where are you going so early, Little Red Cap?"

"To grandmother's."

"And what are you carrying under your apron?"
''')
#open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt', encoding ='utf-8')
i=0
for sent in doc.sents:
    i+=1
    print('i=', str(i)+sent.text)

i= 1
Once upon a time there was a sweet little girl.
i= 2Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next.
i= 3Once she gave her a little cap made of red velvet.
i= 4Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Cap.

i= 5One day her mother said to her, "Come Little Red Cap.
i= 6Here is a piece of cake and a bottle of wine.
i= 7Take them to your grandmother.
i= 8She is sick and weak, and they will do her well.
i= 9Mind your manners and give her my greetings.
i= 10Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother.
i= 11"

Little Red Cap promised to obey her mother.
i= 12The grandmother lived out in the woods, a half hour from the village.
i= 13When Little Red Cap entered the woods a wolf came up to her.
i= 14She did not know what a wicked animal he was, and was no

In [46]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

She she PRON PRP nsubj Xxx True True
did do AUX VBD aux xxx True True
not not PART RB neg xxx True True
know know VERB VB ROOT xxxx True False
what what PRON WP det xxxx True True
a a DET DT det x True True
wicked wicked ADJ JJ amod xxxx True False
animal animal NOUN NN npadvmod xxxx True False
he he PRON PRP nsubj xx True True
was be AUX VBD ccomp xxx True True
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
was be AUX VBD conj xxx True True
not not PART RB neg xxx True True
afraid afraid ADJ JJ acomp xxxx True False
of of ADP IN prep xx True True
him he PRON PRP pobj xxx True True


In [32]:
displacy.render(doc, style='dep', jupyter=True)

'do'

In [41]:
token = doc
print(token.text)
for word in doc:
    print('lemma:', word.lemma_)

She did not know what a wicked animal he was, and was not afraid of him
lemma: she
lemma: do
lemma: not
lemma: know
lemma: what
lemma: a
lemma: wicked
lemma: animal
lemma: he
lemma: be
lemma: ,
lemma: and
lemma: be
lemma: not
lemma: afraid
lemma: of
lemma: he


In [24]:
nlp = Russian()
doc = nlp("Съешь ещё этих мягких французских булок, да выпей чаю.")

In [25]:
token = doc[0]
print(token.text)

span = doc[3:6]
print(span.text)

Съешь
мягких французских булок


In [52]:
print("is_alpha:    ", [token.is_alpha for token in doc])
print("is_punct:    ", [token.is_punct for token in doc])
print("like_num:    ", [token.like_num for token in doc])

is_alpha:     [True, True, True, True, True, True, False, True, True, True, False]
is_punct:     [False, False, False, False, False, False, True, False, False, False, True]
like_num:     [False, False, False, False, False, False, False, False, False, False, False]


### Gensim

In [23]:
import gensim
import os
from gensim.utils import simple_preprocess
  
#  прочитать текстовый файл как объект
doc = open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt', encoding ='utf-8')
  
# предварительная обработка файла для получения списка токенов
tokenized = []
for sentence in doc.read().split('.'):
    # функция simple_preprocess возвращает список слов каждого предложения
    tokenized.append(simple_preprocess(sentence, deacc = True))
  
for i in tokenized:
    print(i)
doc.close()

['little', 'red', 'cap', 'jacob', 'and', 'wilhelm', 'grimm', 'once', 'upon', 'time', 'there', 'was', 'sweet', 'little', 'girl']
['everyone', 'who', 'saw', 'her', 'liked', 'her', 'but', 'most', 'of', 'all', 'her', 'grandmother', 'who', 'did', 'not', 'know', 'what', 'to', 'give', 'the', 'child', 'next']
['once', 'she', 'gave', 'her', 'little', 'cap', 'made', 'of', 'red', 'velvet']
['because', 'it', 'suited', 'her', 'so', 'well', 'and', 'she', 'wanted', 'to', 'wear', 'it', 'all', 'the', 'time', 'she', 'came', 'to', 'be', 'known', 'as', 'little', 'red', 'cap']
['one', 'day', 'her', 'mother', 'said', 'to', 'her', 'come', 'little', 'red', 'cap']
['here', 'is', 'piece', 'of', 'cake', 'and', 'bottle', 'of', 'wine']
['take', 'them', 'to', 'your', 'grandmother']
['she', 'is', 'sick', 'and', 'weak', 'and', 'they', 'will', 'do', 'her', 'well']
['mind', 'your', 'manners', 'and', 'give', 'her', 'my', 'greetings']
['behave', 'yourself', 'on', 'the', 'way', 'and', 'do', 'not', 'leave', 'the', 'path', 

### Pyinflect чтобы изменять слова (времена глагола, степени прилагательных, число множ/ед)

In [31]:
import pyinflect
nlp = spacy.load('en_core_web_sm')
tokens = nlp('This is an example of xxtest.')
tokens[3]._.inflect('NNS')

'examples'

In [45]:
for i in range(len(tokens)):
    # тут условие if проверка на часть речи
    for j in range(0, 3):
        print(tokens[i]._.inflect('VBD', form_num=j))

None
None
None
was
were
was
None
None
None
exampled
exampled
exampled
None
None
None
None
None
None
None
None
None


In [33]:
tokens[1]._.inflect('VBD', form_num=1)

'were'

In [34]:
tokens[5]._.inflect('VBG', inflect_oov=True)

'xxtesting'

### Bag of word (BOW)

In [14]:
df_sentences

,sentence
0,Once upon a time there was a sweet little girl.
1,"Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next."
2,Once she gave her a little cap made of red velvet.
3,"Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Cap."
4,"One day her mother said to her, ""Come Little Red Cap."
5,Here is a piece of cake and a bottle of wine.
6,Take them to your grandmother.
7,"She is sick and weak, and they will do her well."
8,Mind your manners and give her my greetings.
9,"Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother."""


In [20]:
allsentences = ["Joe waited for the train train", "The train was late", "Mary and Samantha took the bus"]

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(allsentences)
print(X.toarray())

[[0 0 1 1 0 0 0 1 0 2 1 0]
 [0 0 0 0 1 0 0 1 0 1 0 1]
 [1 1 0 0 0 1 1 1 1 0 0 0]]


In [29]:
df_sentences.sentence.values.tolist()

['Once upon a time there was a sweet little girl.',
 'Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next.',
 'Once she gave her a little cap made of red velvet.',
 'Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Cap.',
 'One day her mother said to her, "Come Little Red Cap.',
 'Here is a piece of cake and a bottle of wine.',
 'Take them to your grandmother.',
 'She is sick and weak, and they will do her well.',
 'Mind your manners and give her my greetings.',
 'Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother."',
 'Little Red Cap promised to obey her mother.',
 'The grandmother lived out in the woods, a half hour from the village.',
 'When Little Red Cap entered the woods a wolf came up to her.',
 'She did not know what a wicked animal he was, and was not afraid of

In [33]:
X

<3x12 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>